In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_chuko_mansions():
    base_url = 'https://www.homes.co.jp/mansion/chuko/tokyo/23ku/list/'
    max_page = 5  # 最大ページ数
    mansions = []

    for page in range(1, max_page):
        url = f'{base_url}?page={page}'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        for mansion in soup.select('.moduleInner'):
            try:
                name = mansion.select_one('.bukkenName').text.strip()

                # 所在地を取得
                location_parts = mansion.select('td')[0].text.strip()

                # 価格を取得
                price_text = mansion.select_one('.priceLabel').text.strip()
                price = int(price_text.replace('万円', '').replace(',', '')) 

                # 間取りと専有面積を分けて取得
                layout_and_area = mansion.select_one('.layoutSpace').text.strip()
                layout, area = layout_and_area.split('/')

                # 築年数を取得
                construction_year_text = mansion.select('td')[2].text.strip()
                construction_year = construction_year_text.split('（')[1].split('）')[0]

                mansions.append({
                    'name': name,
                    'location_parts': location_parts,
                    'price': price,
                    'layout': layout.strip(),
                    'area': area.strip(),
                    'construction_year': construction_year
                })
            except AttributeError:
                continue

    df = pd.DataFrame(mansions)
    return df

# マンション情報を取得
mansion_data = scrape_chuko_mansions()
print(mansion_data)


                        name                        location_parts  price  \
0                  ナイスアーバン浅草      東武伊勢崎線 浅草駅 徒歩11分東京都台東区東浅草1丁目1-17   6180   
1                  カーサフェリス洗足        東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16   7990   
2           ライオンズマンション西麻布シティ    東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8  11390   
3                イニシアイオ新宿夏目坂      都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3   9580   
4                百花園パーク・ホームズ      東武伊勢崎線 東向島駅 徒歩8分東京都墨田区東向島3丁目28-1   5280   
..                       ...                                   ...    ...   
115                ミルーナヒルズ舎人        日暮里舎人ライナー 舎人駅 徒歩13分東京都足立区入谷2丁目   3300   
116  クレイシア・ヴァースクレイシア 江戸川橋レーヴ  東京メトロ有楽町線 江戸川橋駅 徒歩8分東京都文京区水道2丁目17-12   6190   
117                 ピアース麻布十番        東京メトロ南北線 麻布十番駅 徒歩7分東京都港区南麻布1丁目   5600   
118              デュオステージ新高円寺  東京メトロ丸ノ内線 新高円寺駅 徒歩3分東京都杉並区梅里1丁目13-17   4890   
119   ミルーナヒルズ舎人Ｃ’ｓＣＯＬＬＥＣＴＩＯＮ    日暮里舎人ライナー 舎人駅 徒歩14分東京都足立区入谷2丁目23-8   3180   

    layout     area construction_year  
0     3LDK  58.19m²              築2

In [13]:
import re

# 正規表現を使用して情報を抽出
mansion_data["transportation"] = mansion_data["location_parts"].apply(lambda x: re.search(r"(.*) 徒歩(\d+)分", x).group(1) if isinstance(x, str) else None)
mansion_data["minutes"] = mansion_data["location_parts"].apply(lambda x: re.search(r"徒歩(\d+)分", x).group(1) if isinstance(x, str) else None)
mansion_data["address"] = mansion_data["location_parts"].apply(lambda x: re.search(r"徒歩\d+分(.*)", x).group(1) if isinstance(x, str) else None)

# 「m²」を削除して数値に変換
mansion_data['area'] = mansion_data['area'].apply(lambda x: float(x.replace('m²', '')) if isinstance(x, str) else None)

# 所在地から「東京都」の後から「○○区」までを抜き出す関数
def extract_district(address):
    match = re.search(r'東京都(.+?区)', address)
    if match:
        return match.group(1)
    else:
        return None

# 区という項目に出力
mansion_data['区'] = mansion_data['address'].apply(extract_district)

# 正規表現パターン（キャプチャグループを追加）
pattern = r"築(\d+)年"

# 築年数から数字のみを抽出して修正
mansion_data['construction_year'] = mansion_data['construction_year'].str.extract(pattern).astype(int)

mansion_data

,name,location_parts,price,layout,area,construction_year,transportation,minutes,address,区
0,ナイスアーバン浅草,東武伊勢崎線 浅草駅 徒歩11分東京都台東区東浅草1丁目1-17,6180,3LDK,58.19,25,東武伊勢崎線 浅草駅,11,東京都台東区東浅草1丁目1-17,台東区
1,カーサフェリス洗足,東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16,7990,3LDK,60.32,22,東急目黒線 洗足駅,10,東京都品川区荏原7丁目9-16,品川区
2,ライオンズマンション西麻布シティ,東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8,11390,2LDK,56.46,25,東京メトロ日比谷線 六本木駅,9,東京都港区西麻布2丁目13-8,港区
3,イニシアイオ新宿夏目坂,都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3,9580,2LDK,59.13,15,都営大江戸線 若松河田駅,6,東京都新宿区原町3丁目64-3,新宿区
4,百花園パーク・ホームズ,東武伊勢崎線 東向島駅 徒歩8分東京都墨田区東向島3丁目28-1,5280,3LDK,71.98,39,東武伊勢崎線 東向島駅,8,東京都墨田区東向島3丁目28-1,墨田区
...,...,...,...,...,...,...,...,...,...,...
115,ミルーナヒルズ舎人,日暮里舎人ライナー 舎人駅 徒歩13分東京都足立区入谷2丁目,3300,1LDK,43.70,5,日暮里舎人ライナー 舎人駅,13,東京都足立区入谷2丁目,足立区
116,クレイシア・ヴァースクレイシア 江戸川橋レーヴ,東京メトロ有楽町線 江戸川橋駅 徒歩8分東京都文京区水道2丁目17-12,6190,1LDK,40.46,5,東京メトロ有楽町線 江戸川橋駅,8,東京都文京区水道2丁目17-12,文京区
117,ピアース麻布十番,東京メトロ南北線 麻布十番駅 徒歩7分東京都港区南麻布1丁目,5600,ワンルーム,25.22,5,東京メトロ南北線 麻布十番駅,7,東京都港区南麻布1丁目,港区
118,デュオステージ新高円寺,東京メトロ丸ノ内線 新高円寺駅 徒歩3分東京都杉並区梅里1丁目13-17,4890,1SLDK,40.95,5,東京メトロ丸ノ内線 新高円寺駅,3,東京都杉並区梅里1丁目13-17,杉並区


In [31]:
#import gspread
#from gspread_dataframe import set_with_dataframe

# Google Sheets の認証
#gc = gspread.service_account(filename='spread-sheet-api-419202-59ee0bee5075.json')  # 認証情報ファイルのパスを指定

# スプレッドシートを開く（スプレッドシートの URL から ID を取得して指定）
#spreadsheet_id = '1qjYieNxp0gG3fOuDNkV46DL4Q1wK8vn_kcZTiYwtVIw'
#worksheet_name = '購入物件'
#sh = gc.open_by_key(spreadsheet_id)
#worksheet = sh.worksheet(worksheet_name)

# DataFrame をスプレッドシートに書き込む
#set_with_dataframe(worksheet, mansion_data)

In [15]:
import sqlite3

# SQLite データベースに接続
conn = sqlite3.connect('chuko_database.db')
cursor = conn.cursor()

# 既存のテーブルを削除
cursor.execute('DROP TABLE IF EXISTS chuko_table')

# テーブル作成
cursor.execute('CREATE TABLE chuko_table (id INTEGER PRIMARY KEY, name TEXT, location_parts TEXT, price_range INTEGER, layout_types TEXT, area REAL, age_range TEXT, transportation TEXT, walk_distance INTEGER, address TEXT, selected_districts TEXT,latitude REAL,longitude REAL)')

# データ挿入
for _, row in mansion_data.iterrows():
    cursor.execute('INSERT INTO chuko_table (name, location_parts, price_range, layout_types, area, age_range, transportation, walk_distance, address, selected_districts) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', 
                   (row['name'], row['location_parts'], row['price'], row['layout'], row['area'], row['construction_year'], row['transportation'], row['minutes'], row['address'], row['区']))

# コミットと接続のクローズ
conn.commit()
conn.close()

print("データベースにデータを保存しました。")


データベースにデータを保存しました。
